## 1) Ortam Kurulumu ve Tekrarlanabilirlik

Bu hücre, derin öğrenme projelerine başlamadan önce **çalışma ortamını hazırlar ve sonuçların tekrarlanabilir olmasını sağlar.**

- **Kütüphaneler**: `os`, `random`, `numpy`, `tensorflow` modülleri içe aktarılır.  
- **Log düzeni**: `os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"` ile TensorFlow’un gereksiz uyarı ve bilgi mesajları gizlenir.  
- **Rastgelelik sabitleme**: `random.seed`, `np.random.seed` ve `tf.random.set_seed` komutları ile tüm rastgele işlemler aynı başlangıç değerine bağlanır. Bu sayede her çalıştırmada aynı sonuçlar alınabilir (**reproducibility**).  
- **GPU bellek yönetimi**: `tf.config.experimental.set_memory_growth` ile TensorFlow GPU belleğini ihtiyaç oldukça kullanır, ilk başta tamamını rezerve etmez. Bu, bellek taşma (OOM) hatalarını önlemeye yardımcı olur.  
- **Durum bildirimi**: Mevcut fiziksel GPU sayısı ekrana yazdırılır; `0` görülürse işlem CPU üzerinden yapılır.

> Bu hücre sayesinde **temiz, düzenli ve öngörülebilir bir çalışma ortamı** oluşturulur.


In [ ]:
import os, random, numpy as np, tensorflow as tf

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)
gpus = tf.config.list_physical_devices('GPU')
for g in gpus:
    try: tf.config.experimental.set_memory_growth(g, True)
    except: pass
print("GPU sayısı:", len(gpus))



## 2) Veri Yollarının Belirlenmesi ve Keşif (Data Paths & Exploration)

Bu hücrede **görüntü verisinin yolu belirlenir, dosyalar listelenir ve eğitim/validasyon/test ayrımı yapılır.**

- **Kütüphaneler**:  
  - `pandas`: Veri çerçevesi (DataFrame) oluşturma ve düzenleme için.  
  - `pathlib.Path`: Dosya yollarını platformdan bağımsız biçimde yönetmek için.  

- **Veri konumu**: `DATA_DIR` ile Kaggle’ın *flowers-recognition* veri kümesindeki kök klasör tanımlanır. Alt klasörler çiçek sınıflarını (örneğin *rose*, *daisy*) temsil eder.

- **Dosya listesi çıkarma**:  
  - `glob` yöntemi ile `*.jpg`, `*.jpeg`, `*.png` uzantılı tüm görüntüler bulunur.  
  - Her bir dosyanın tam yolu `paths`, dosyanın klasör adı ise (etiket) `labels` listesine kaydedilir.

- **Veri çerçevesi**: `df` adlı DataFrame’de her satırda bir görüntü yolu ve buna karşılık gelen etiket bulunur.  
  - `sample(frac=1, random_state=SEED)` verileri karıştırır.  
  - `reset_index(drop=True)` ile yeni bir düzenli indeks oluşturulur.

- **Eğitim / Doğrulama / Test bölünmesi**:  
  - `train_test_split` ile verinin %15’i test kümesi olarak ayrılır.  
  - Kalan %85’in içinden tekrar bölünerek yaklaşık %15’i validasyon için ayrılır.  
  - `stratify` parametresi, her sınıftaki oranın tüm alt kümelerde korunmasını sağlar.

- **Sınıf bilgisi**:  
  - `classes`: Tüm çiçek sınıflarının alfabetik listesi.  
  - `class_to_idx`: Her sınıfa bir numara atayan sözlük.  
  - `num_classes`: Toplam sınıf sayısı.

- **Kontrol çıktıları**:  
  - Toplam görüntü sayısı (`len(all_images)`).  
  - Bulunan sınıflar listesi (`classes`).

> Bu hücre sayesinde **ham görsel veriler düzenlenir, etiketlenir ve model eğitimine hazır hale gelir**.


In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("/kaggle/input/flowers-recognition/flowers") 
all_images = []
for ext in ("*.jpg","*.jpeg","*.png"): all_images += list(DATA_DIR.glob(f"*/*{ext[1:]}"))
print("Görüntü sayısı:", len(all_images))

paths  = [str(p) for p in all_images]
labels = [p.parent.name for p in all_images]
df = pd.DataFrame({"path": paths, "label": labels}).sample(frac=1, random_state=SEED).reset_index(drop=True)

from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.15, random_state=SEED, stratify=df["label"])
train_df, val_df  = train_test_split(train_df, test_size=0.15/(1-0.15), random_state=SEED, stratify=train_df["label"])

classes = sorted(df["label"].unique())
class_to_idx = {c:i for i,c in enumerate(classes)}
num_classes = len(classes)
print("Sınıflar:", classes)

## 3) Eğitim Verisinin Sınıf Dağılımı

Bu hücre, eğitim kümesindeki her çiçek sınıfının kaç örneğe sahip olduğunu **çubuk grafik** ile gösterir.

- `value_counts().plot(kind="bar")` → Her sınıfın görüntü sayısını bar grafiğe döker.  
- `figsize`, `color`, `edgecolor` → Grafik boyutu ve renk ayarları.  
- `set_title`, `set_xlabel`, `set_ylabel` → Başlık ve eksen isimleri.  
- `xticks(rotation=45)` → Sınıf adlarını eğerek okunabilirlik sağlar.

> Veri dengesini hızlıca görmek ve olası dengesizlikleri fark etmek için kullanılır.


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8,5))
train_df["label"].value_counts().plot(kind="bar", ax=ax, color="skyblue", edgecolor="black")
ax.set_title("Train Set – Sınıf Dağılımı")
ax.set_xlabel("Sınıf")
ax.set_ylabel("Görüntü Sayısı")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 4) Eğitim Setinden Rastgele Görseller

Bu hücre, eğitim kümesinden **rastgele 15 görseli** tablo şeklinde gösterir.

- `random.sample` → Eğitim verisinden 15 rastgele dosya yolu seçer.  
- `Image.open` → Her görüntüyü okur ve ekranda gösterir.  
- `plt.subplots(3,5)` → 3 satır 5 sütunlu bir figür alanı oluşturur.  
- `ax.imshow` → Görüntüyü ilgili eksene çizer.  
- `ax.set_title` → Her görselin üstüne sınıf adını (etiketini) yazar.  
- `ax.axis("off")` → Eksen çizgilerini kaldırır.  
- `plt.suptitle` ve `plt.tight_layout` → Genel başlık ekler, kenar boşluklarını düzenler.

> Veri setinin içeriğini hızlıca gözden geçirip **görsellerin çeşitliliğini ve etiketlerin doğruluğunu** kontrol etmek için kullanılır.


In [ ]:
import random
from PIL import Image

fig, axes = plt.subplots(3, 5, figsize=(12,7))
sample_paths = random.sample(train_df["path"].tolist(), 15)

for ax, img_path in zip(axes.flat, sample_paths):
    img = Image.open(img_path)
    ax.imshow(img)
    ax.set_title(Path(img_path).parent.name, fontsize=8)
    ax.axis("off")

plt.suptitle("Train Set – Rastgele 15 Görsel", fontsize=14)
plt.tight_layout()
plt.show()


## 5) Görsel Veri Seti Hazırlığı ve Veri Artırma (Data Augmentation)

Bu hücre, resimleri **modele uygun formata çevirir** ve eğitim sırasında **veri artırma (data augmentation)** ile modeli genelleştirme gücü yüksek hale getirir.

### 5.1) Temel Ayarlar
- **IMG_SIZE = 224** → EfficientNet gibi çoğu önceden eğitilmiş modelin beklediği standart giriş boyutu.  
- **BATCH = 32** → Her eğitim adımında işlenecek görüntü sayısı.  
- **AUTO = tf.data.AUTOTUNE** → TensorFlow, veri yükleme ve önbellekleme işlerini otomatik en iyi performansla yürütür.

### 5.2) Görseli Okuma ve Ön İşleme
`decode_and_resize(path, label_idx)`  
- Dosya yolundaki resmi okur (`tf.io.read_file`), RGB kanallarına çevirir (`decode_image`).
- Boyutunu 224×224 piksele yeniden boyutlandırır (`tf.image.resize`).
- Piksel değerlerini `float32` tipine çevirir ve `preprocess_input` ile **[-1,1] aralığına** normalize eder.
- Etiketi `one-hot` formuna dönüştürür (örneğin 5 sınıf varsa `[0,1,0,0,0]` gibi).

### 5.3) Veri Artırma Katmanı (Data Augmentation)
`aug` → `keras.Sequential` ile oluşturulan dönüşümler:
- `layers.RandomFlip(mode="horizontal")` → Görüntüyü rastgele yatay çevirir.  
- `layers.RandomRotation(factor=0.08)` → Maksimum ±%8 oranında döndürür.  
- `layers.RandomZoom(height_factor=0.1, width_factor=0.1)` → Yükseklik ve genişlikte %10’a kadar yakınlaştırır.  
- `layers.RandomContrast(factor=0.1)` → Kontrastı %10 oranında değiştirir.  

> **Amaç**: Modelin tek tip veriye aşırı uyum sağlamasını (overfitting) önlemek, gerçek dünyadaki konum, ışık ve poz değişikliklerine karşı **daha dayanıklı** hale getirmek.

### 5.4) Veri Kümesi Oluşturma
`make_ds(frame, training=False)`  
- DataFrame’den dosya yolları ve etiketleri alır.  
- Eğitim modundaysa veriyi her epoch’ta rastgele karıştırır.  
- Görselleri `decode_and_resize` ile işler, eğitim modunda `augment_if_training` ile veri artırır.  
- Veriyi `batch` ve `prefetch` ile hızlandırır.

### 5.5) Sonuç Dataset’leri
- `train_ds` → Veri artırma uygulanmış eğitim kümesi.  
- `val_ds`   → Doğrulama (modeli değerlendirirken veri artırma yok).  
- `test_ds`  → Test kümesi.

> Bu adım, görselleri **standart boyut ve ölçeğe getirir**, veri artırma ile çeşitlendirir ve modeli **daha kararlı ve genelleme yeteneği yüksek** hale getirir.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers

IMG_SIZE = 224
BATCH    = 32
AUTO     = tf.data.AUTOTUNE

def decode_and_resize(path, label_idx):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE), method=tf.image.ResizeMethod.BILINEAR)
    img = tf.cast(img, tf.float32)
    img = preprocess_input(img)   # [-1,1]
    y = tf.one_hot(label_idx, depth=num_classes)
    return img, y

aug = tf.keras.Sequential(
    [
        layers.RandomFlip(mode="horizontal"),     
        layers.RandomRotation(factor=0.08),        
        layers.RandomZoom(height_factor=0.1, width_factor=0.1),
        layers.RandomContrast(factor=0.1),
    ], name="augment")

def augment_if_training(img, y):
    return aug(img, training=True), y

def make_ds(frame, training=False):
    paths = frame["path"].values
    labs  = frame["label"].map(class_to_idx).values
    ds = tf.data.Dataset.from_tensor_slices((paths, labs))
    if training: ds = ds.shuffle(len(frame), seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(decode_and_resize, num_parallel_calls=AUTO)
    if training: ds = ds.map(augment_if_training, num_parallel_calls=AUTO)
    ds = ds.batch(BATCH).prefetch(AUTO)
    return ds

train_ds = make_ds(train_df, training=True)
val_ds   = make_ds(val_df,   training=False)
test_ds  = make_ds(test_df,  training=False)

## 6) Veri Artırma (Augmentation) Sonuçlarının Görselleştirilmesi

Bu hücre, eğitim kümesinden alınan bir batch (toplu) veride **uygulanmış rastgele artırma işlemlerini** görsel olarak gösterir.

- `train_ds.take(1)` → Eğitim veri kümesinden tek bir batch çeker.  
- `x_aug` → Veri artırma uygulanmış görüntüler; `numpy().astype(np.uint8)` ile görüntü formatına dönüştürülür.  
- `k = min(8, x_show.shape[0])` → İlk 8 görüntü seçilir.  
- `plt.subplot(2,4,i+1)` ve `plt.imshow` → 2 satır 4 sütunlu bir tabloya her görüntü yerleştirilir.  
- `plt.axis('off')` → Eksen çizgileri kaldırılır.  
- `plt.suptitle` ve `plt.tight_layout` → Genel başlık ekler, düzenli görünüm sağlar.

> Bu görselleştirme ile **RandomFlip, RandomRotation, RandomZoom, RandomContrast** gibi veri artırma adımlarının resimlere nasıl uygulandığı doğrudan görülebilir.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for x_aug, yb in train_ds.take(1):
    x_show = x_aug.numpy().astype(np.uint8)
    break

k = min(8, x_show.shape[0])
plt.figure(figsize=(12,6))
for i in range(k):
    plt.subplot(2,4,i+1)
    plt.imshow(x_show[i])
    plt.axis('off')
plt.suptitle("Augmentation örnekleri")
plt.tight_layout()
plt.show()


## 7) Model Kurulumu, Derleme ve Callback’ler

Bu hücre, **EfficientNetB0 tabanlı bir transfer öğrenme modeli** kurar, uygun kayıp/optimizasyon ayarlarıyla derler ve eğitimde kullanılacak **callback**’leri hazırlar.

### 🔩 Mimari (Transfer Learning)
- `EfficientNetB0(include_top=False, weights="imagenet")`  
  - **Ön-eğitimli** (ImageNet) taban; tepe (classification) katmanı çıkarılmıştır.

- Akış:
  1. **Input**: `(224,224,3)`
  2. **base(inputs, training=False)**: BatchNorm dalgalanmalarını önlemek için çıkarım modunda çağrılır.
  3. **GlobalAveragePooling2D**: Özellik haritalarını vektöre indirger, parametre sayısını azaltır.
  4. **Dropout(0.4)**: Aşırı uyumu (overfitting) azaltır.
  5. **Dense(num_classes, softmax)**: Sınıf olasılıklarını üretir.  
     - `l2` verilirse `kernel_regularizer=l2(...)` ile **ağırlık cezalandırma** eklenir.

### 🧪 Derleme (Loss & Optimizer)
- **Loss**: `CategoricalCrossentropy(label_smoothing=...)`  
  - `label_smoothing`: Çok güvenli tahminleri yumuşatır, genellemeye yardımcı olabilir (varsayılan 0.0).
- **Optimizer**:  
  - `"adam"` → `Adam(lr)`: Hızlı yakınsama, genelde iyi varsayılan.  
  - Aksi halde `SGD(lr, momentum=0.9, nesterov=True)`: Daha kontrollü güncellemeler.

### ⏱️ Callback’ler
- `ModelCheckpoint("best_stage1.keras", monitor="val_accuracy", mode="max", save_best_only=True)`  
  - **En iyi doğrulama başarımı** yakalandığında ağırlıkları kaydeder (Stage-1).  
- `EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True)`  
  - İyileşme durunca eğitimi erken durdurur; **en iyi ağırlıklara** geri döner.
- `ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-7)`  
  - Kayıp iyileşmiyorsa öğrenme oranını **yarıya indirir** (plateau’da daha ince ayar).

### ▶️ Stage-1 Eğitimi
- `build_model(dropout=0.4, base_trainable=False)` ile tabanı dondurulmuş model kurulur.  
- `compile_with(model, lr=1e-3, opt="adam")` ile uygun öğrenme oranı ve optimizer ayarlanır.  
- `model.summary()` mimariyi özetler.

In [ ]:

from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

def build_model(dropout=0.4, l2=None, base_trainable=False):
    base = EfficientNetB0(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights="imagenet")
    base.trainable = base_trainable
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="input_preprocessed")
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout)(x)
    kw = {} if l2 is None else {"kernel_regularizer": regularizers.l2(l2)}
    outputs = layers.Dense(num_classes, activation="softmax", **kw)(x)
    return models.Model(inputs, outputs, name="flowers_effnetb0")

def compile_with(model, lr, opt="adam", label_smoothing=0.0):
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing)
    optimizer = tf.keras.optimizers.Adam(lr) if opt=="adam" else tf.keras.optimizers.SGD(lr, momentum=0.9, nesterov=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
    return model

ckpt1 = ModelCheckpoint("best_stage1.keras", monitor="val_accuracy", save_best_only=True, mode="max")
es    = EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True)
rlr   = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-7)
model = build_model(dropout=0.4, base_trainable=False)
compile_with(model, lr=1e-3, opt="adam")
print(model.summary())

## 8) Model Mimarisi Görselleştirme

Bu hücre, kurulmuş olan **derin öğrenme modelinin yapısını resim dosyası olarak kaydeder**.

- `plot_model(model, to_file="model_architecture.png")` → Model mimarisini `model_architecture.png` adıyla kaydeder.  
- `show_shapes=True` → Her katmanın giriş/çıkış boyutlarını gösterir.  
- `show_layer_names=True` → Katman isimlerini diyagrama ekler.


In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file="model_architecture.png",
           show_shapes=True, show_layer_names=True)

## 9) Model Eğitimi

Bu hücre, EfficientNet tabanlı modelin **sınıflandırma katmanlarını eğitir.** 

- **`model.fit` parametreleri**  
  - `train_ds` → Eğitim verisi.  
  - `validation_data=val_ds` → Her epoch sonunda doğrulama kümesinde ölçüm yapılır.  
  - `epochs=15` → Maksimum 15 eğitim turu.  
  - `callbacks=[ckpt1, es, rlr]` →  
    - `ckpt1`: En iyi doğrulama doğruluğunda modeli kaydeder.  
    - `es`: İyileşme durursa erken durdurur ve en iyi ağırlıkları geri yükler.  
    - `rlr`: Doğrulama kaybı iyileşmezse öğrenme oranını otomatik yarıya indirir.
  - `verbose=1` → Her epoch için ilerleme çubuğu ve metrikler gösterilir.

- **`hist1` çıktısı**  
  - Eğitim süresince **kayıp (loss)** ve **doğruluk (accuracy)** değerlerini kaydeder.  
  - Bu bilgiler, sonraki adımda grafik çizerek modelin **öğrenme eğrisini** incelemek için kullanılır.

> Bu aşama, önceden eğitilmiş EfficientNet tabanını değiştirmeden  üst sınıflandırıcı katmanını eğiterek **hızlı ve stabil bir eğitim yapılabilmesini sağlar**.


In [ ]:
hist1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[ckpt1, es, rlr],
    verbose=1
)

## 10) Öğrenme Eğrileri (Kısa, Net, Detaylı)

**Amaç**
- Eğitim/validasyon **accuracy** ve **loss** eğrilerini çizmek, son epoch’lara bakarak **overfitting/plato** sinyallerini otomatik yorumlamak.

**Fonksiyonlar**
- `plot_history(h, title)`:  
  - Solda: `accuracy` (train vs val)  
  - Sağda: `loss` (train vs val)
- `comment_learning_curves(history, tail=5)`:  
  - Son `k=tail` epoch ortalamalarına göre farkları hesaplar:
    - `acc_gap = mean(train_acc - val_acc)`  
      - `> 0.02` → **Overfitting** ihtimali  
      - `≈ 0` → **Genelleme** iyi  
    - `loss_gap = mean(val_loss - train_loss)`  
      - Pozitif ve büyük → **Overfitting** riski
  - **Plato**: Son `k` epoch’ta `val_acc` artışı `< 0.005` ise uyarır.

**Öneri Şablonları (Duruma Göre)**
- Overfitting: **Daha güçlü augmentation**, **Dropout/L2↑**, **LR↓**, **EarlyStopping** sıkılaştır.
- Plato: **ReduceLROnPlateau**, gerekirse **küçük LR ile üst katmanları fine-tune** et.
- Val > Train: Birkaç epoch daha dene; **LR ısınma/plateau** stratejisini gözden geçir.


In [ ]:

import matplotlib.pyplot as plt
import numpy as np


def plot_history(h, title):
    plt.figure(figsize=(12,4))
    # Acc
    plt.subplot(1,2,1)
    plt.plot(h.history["accuracy"], label="train_acc")
    plt.plot(h.history["val_accuracy"], label="val_acc")
    plt.title(f"{title} - Accuracy"); plt.grid(True); plt.legend()
    # Loss
    plt.subplot(1,2,2)
    plt.plot(h.history["loss"], label="train_loss")
    plt.plot(h.history["val_loss"], label="val_loss")
    plt.title(f"{title} - Loss"); plt.grid(True); plt.legend()
    plt.show()

plot_history(hist1, "Stage-1")


def comment_learning_curves(history, tail=5):
    h = history.history
    acc  = np.array(h.get("accuracy", []), dtype=float)
    val_acc = np.array(h.get("val_accuracy", []), dtype=float)
    loss = np.array(h.get("loss", []), dtype=float)
    val_loss = np.array(h.get("val_loss", []), dtype=float)

    if len(val_acc)==0 or len(acc)==0:
        print("Not: accuracy/val_accuracy bulunamadı; yorum atlandı.")
        return

    # Son k epoch ortalamaları & farklar
    k = min(tail, len(acc))
    acc_gap  = float(np.nanmean(acc[-k:] - val_acc[-k:]))   # pozitifse overfit ihtimali
    loss_gap = float(np.nanmean(val_loss[-k:] - loss[-k:])) if len(val_loss)==len(loss)>0 else np.nan

    # Basit plato göstergesi: son k adımda val_acc artışı çok küçükse
    plateau = (np.nanmax(val_acc) - np.nanmin(val_acc[-k:])) < 0.005

    lines = []
    lines.append(f"Özet: Son {k} epoch ortalamasında accuracy farkı (train - val) ≈ {acc_gap:.3f}.")

    if acc_gap > 0.02:
        lines.append("Yorum: Train doğruluğu belirgin şekilde daha yüksek → **overfitting** işareti.")
        lines.append("Öneri: Augmentation’ı güçlendir, Dropout/L2 ekle/artır, LR’ı azalt veya erken durdurmayı sıkılaştır.")
    elif acc_gap < -0.01:
        lines.append("Yorum: Val doğruluğu train’den yüksek görünüyor; muhtemelen data shuffle/batch etkisi.")
        lines.append("Öneri: Birkaç epoch daha eğitip dengele; LR warmup/plateau stratejisini gözden geçir.")
    else:
        lines.append("Yorum: Train/Val eğrileri yakın seyrediyor → **genelleme** kabul edilebilir düzeyde.")

    if plateau:
        lines.append("Not: Val accuracy son dönemde plato yaptı.")
        lines.append("Öneri: ReduceLROnPlateau/TTA veya alt katmanların bir bölümünü açıp küçük LR ile fine-tuning dene.")

    if not np.isnan(loss_gap):
        lines.append(f"Loss farkı (val - train) ≈ {loss_gap:.3f}. Pozitif ve büyükse overfit riski artar.")

    print("\n".join(lines))

comment_learning_curves(hist1, tail=5)



## 11) Eğitim Özeti – En İyi Epoch ve Son Değerler

Bu hücre, her eğitim aşamasının **en iyi doğrulama başarımını ve son metriklerini kısaca raporlar.**

**Fonksiyon**
- `summarize_curves(history)`:
  - `best_val_acc` : En yüksek doğrulama doğruluğu (val_accuracy) ve gerçekleştiği epoch.
  - `last_train_acc` : Son epoch’taki eğitim doğruluğu.
  - `last_val_loss` : Son epoch’taki doğrulama kaybı.
  - Yorum: Eğer **train_acc ≫ val_acc** ise **overfitting** riski vurgulanır. Ayrıca LR düşüşü (ReduceLROnPlateau) sonrası val_loss’un iyileşmesi olumlu olarak not edilir.


In [ ]:
def summarize_curves(history):
    import numpy as np
    h = history.history
    best_val_acc = float(np.max(h.get("val_accuracy", [0.0])))
    best_val_epoch = int(np.argmax(h.get("val_accuracy", [0.0]))) + 1
    last_train_acc = float(h.get("accuracy", [0.0])[-1]) if "accuracy" in h else float('nan')
    last_val_loss  = float(h.get("val_loss", [0.0])[-1])  if "val_loss"  in h else float('nan')
    print(
        f"[Özet] En iyi val_acc: {best_val_acc:.4f} (epoch {best_val_epoch}) | "
        f"Son train_acc: {last_train_acc:.4f} | Son val_loss: {last_val_loss:.4f}"
    )

summarize_curves(hist1)

## 12) Test Değerlendirme (Tam Sürüm)

Bu hücre, **test kümesi performansını** ayrıntılı raporlar ve **confusion matrix** görselleştirir.

**Adımlar**
- **Gerçek etiketler (`y_true`)**: `test_ds` batch’lerindeki **one-hot** etiketler `argmax` ile sınıf indeksine çevrilerek tek dizide birleştirilir.
- **Tahminler (`y_pred`)**: `model.predict(test_ds)` çıktısından **en yüksek olasılıklı sınıf** `argmax` ile seçilir (isteğe bağlı: `y_pred_probs` analiz için saklanır).
- **Sınıflandırma raporu**: `classification_report` → **precision, recall, f1-score** ve support (sınıf başına örnek sayısı).
- **Confusion matrix**: `confusion_matrix` ile üretilir; sıcaklık haritası olarak çizilir ve hücre içine sayılar yazılır.
  - `thr = cm.max() / 2` → Metin rengi kontrast için otomatik seçilir (büyük hücrelerde **beyaz**, küçüklerde **siyah**).

**Yorumlama İpuçları**
- **Diagonal (True=Pred)** hücreler yüksekse model doğru sınıfları iyi yakalıyor demektir.
- Aynı satırda **yanlış tahmin edilen sütunlar** → o gerçek sınıfın karıştığı sınıfları gösterir (sistematik hata/benzer sınıflar).
- **Macro vs Weighted F1**: Sınıf dengesizliği varsa `weighted f1` daha temsilî, `macro f1` sınıfları **eşit ağırlıklı** değerlendirir.
- Düşük kalan sınıflar için: **augmentation çeşitliliği**, **sınıf ağırlıkları** veya **daha fazla veri** değerlendirilebilir.

**Çıktılar**
- Konsolda ayrıntılı metrik tablosu.
- Etiket isimleriyle döndürülmüş eksenli, üstünde değer yazan **confusion matrix** grafiği.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import seaborn as sns

# --- y_true: batch'lerden toplanır ---
y_true_batches = []
for _, y in test_ds:
    y_true_batches.append(np.argmax(y.numpy(), axis=1))
y_true = np.concatenate(y_true_batches, axis=0)

# --- y_pred: model tahmini ---
y_pred_probs = model.predict(test_ds, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

# --- Rapor ---
print(classification_report(y_true, y_pred, target_names=classes, digits=4))

# --- Confusion Matrix ---
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt="d", cbar=False, 
            xticklabels=classes, yticklabels=classes)
plt.ylabel("Gerçek")
plt.xlabel("Tahmin")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()


## 13) Final Test Accuracy (Tek Satır Sonuç)

Bu hücre, **test kümesindeki genel doğruluk (accuracy)** değerini tek satırda verir.

- `y_true_vec` ve `y_pred_vec` : Gerçek ve tahmin sınıf indeksleri (tek boyutlu vektörler).  
- `(y_true_vec == y_pred_vec).mean()` : Doğru tahmin oranını hesaplar.  
- `print(f"Test Accuracy: {test_acc:.4f}")` : Dört ondalık hassasiyetle ekrana yazdırır.

> Bu metrik, projenin **nihai başarı yüzdesini** hızlıca özetler ve raporlarda temel performans göstergesi olarak kullanılabilir.


In [ ]:
import numpy as np

y_true_vec = np.asarray(y_true).squeeze()
y_pred_vec = np.asarray(y_pred).squeeze()

test_acc = (y_true_vec == y_pred_vec).mean()
print(f"Test Accuracy: {test_acc:.4f}")


## 14) Eigen-CAM ile Görselleştirme

Bu hücre, **modelin karar verirken görüntünün hangi bölgelerine odaklandığını** gösterir. Grad-CAM’e benzer, ancak **özdeğer (eigen) ayrışımı** ile en baskın özellikleri yakalar.

**Adımlar**
- `base = model.get_layer("efficientnetb0")` → EfficientNetB0 taban katmanını alır.
- `_to_uint8_rgb` → Görüntüyü `[0,255]` aralığında `uint8` formatına çevirir.
- `eigen_cam_batch` fonksiyonu:
  1. Özellik haritalarını (`feats`) çıkarır.
  2. Kovaryans matrisinin en büyük özdeğerine karşılık gelen vektörle **en baskın özellik yönünü** bulur.
  3. Bu yönü ısı haritasına (heatmap) dönüştürür.
  4. Orijinal görüntü üzerine yarı saydam olarak bindirir (`alpha=0.35`).

**Kullanım**
- `for xb, yb in test_ds.take(1)` → Test setinden ilk batch’i alır, ilk 5 görüntüyü işler.
- Yan yana iki görsel çizilir:
  - **Input**: Orijinal resim.
  - **Eigen-CAM**: Modelin en çok dikkate aldığı alanları gösteren ısı haritası.

> Bu yöntem, modelin **hangi piksellerden öğrenme yaptığı** ve **hangi bölgelere odaklandığı** hakkında sezgisel ve görsel bir açıklama sunar. 


In [ ]:

import tensorflow as tf, numpy as np, cv2, matplotlib.pyplot as plt

try:
    base 
except NameError:
    base = model.get_layer("efficientnetb0") 


def _to_uint8_rgb(img_tf):
    """img_tf: tf.Tensor (H,W,3) float/uint8; range [-1,1] veya [0,1] veya [0,255] olabilir.
       return: np.uint8 [0,255], RGB
    """
    x = img_tf.numpy()
    if x.dtype != np.uint8:
        x = x.astype(np.float32)
        vmin, vmax = x.min(), x.max()
       
        if vmax <= 1.0 and vmin >= -1.0:     
            if vmin < 0:  # [-1,1]
                x = (x + 1.0) * 127.5
            else:         # [0,1]
                x = x * 255.0
    
        x = np.clip(x, 0, 255)
    return x.astype(np.uint8)  

def eigen_cam_batch(x_imgs, alpha=0.35):
    feats = base(x_imgs, training=False)        
    B, Hc, Wc, C = feats.shape
    outs = []

    for b in range(B):
        F   = feats[b]                       
        F2d = tf.reshape(F, [-1, C])          
        X   = F2d - tf.reduce_mean(F2d, axis=0, keepdims=True)
        cov = tf.matmul(X, X, transpose_a=True) / tf.cast(tf.shape(X)[0]-1, tf.float32)
        _, eigvecs = tf.linalg.eigh(cov)
        w   = eigvecs[:, -1]                   
        h   = tf.reshape(tf.matmul(X, w[:, None]), [Hc, Wc])
        h   = tf.maximum(h, 0)
        h   = (h - tf.reduce_min(h)) / (tf.reduce_max(h) - tf.reduce_min(h) + 1e-12)

        img = _to_uint8_rgb(x_imgs[b])          

        heat = cv2.resize(h.numpy(), (img.shape[1], img.shape[0]))
        heat = (255*heat).astype(np.uint8)
        heat = cv2.applyColorMap(heat, cv2.COLORMAP_JET)          
        heat = cv2.cvtColor(heat, cv2.COLOR_BGR2RGB)           
        vis  = cv2.addWeighted(heat, alpha, img, 1-alpha, 0)

        outs.append((img, vis))
    return outs

for xb, _ in test_ds.take(1):
    arr = xb[0].numpy()
    print("dtype:", arr.dtype, "min/max:", arr.min(), arr.max())

# --- ÖRNEK KULLANIM (test setinden 5 görsel) ---
for xb, yb in test_ds.take(1):
    k = min(5, xb.shape[0])
    res = eigen_cam_batch(xb[:k])
    for i, (img, vis) in enumerate(res):
        plt.figure(figsize=(5,3))
        plt.subplot(1,2,1); plt.imshow(img); plt.axis('off'); plt.title('Input')
        plt.subplot(1,2,2); plt.imshow(vis); plt.axis('off'); plt.title('Eigen-CAM')
        plt.tight_layout(); plt.show()


## 15) Grad-CAM ile Açıklanabilirlik (Base + Head Manuel Zincir)

Bu hücre, **base özellik haritası** üzerinden **head’i (sınıflandırıcı kısmı)** elle çağırarak Grad-CAM ısı haritası üretir. Böylece `model.input` sorunlu/None olsa bile çalışır; Eigen-CAM ile aynı batch (`xb`) üzerinde kullanılabilir.

### Ana Fikir
- **Base (EfficientNetB0)**: Konvolüsyonel özellik haritalarını üretir.
- **Head (GAP + Dropout + Dense)**: Sınıf olasılıklarını hesaplar. Burada head’i **katman katman** kendimiz uygularız.
- **Grad-CAM**: Hedef sınıf skorunun, son konvolüsyon çıkışına göre gradyanlarını kullanarak **kanal ağırlıkları** çıkarır ve ısı haritası üretir.

### Bileşenler
- `_apply_head_from_base_features(feats)`  
  - `model.layers` içinde `base` katmanından **sonra gelen** katmanları sırayla `feats` üzerine uygular.  
  - `InputLayer` atlanır; `training=False` verilerek Dropout vb. deterministik çalışır.  
  - Dönen çıktı: `(1, num_classes)` (veya eşdeğeri).
- `grad_cam_batch(x_imgs, class_idx=None, alpha=0.35)`  
  - Her örnek için:  
    1) `conv_out = base(x1, training=False)`  
    2) `preds = _apply_head_from_base_features(conv_out)`  
    3) Hedef skor (`score`):  
       - `class_idx=None` → **modelin tahmin ettiği sınıf**  
       - `class_idx=int/list[int]` → **belirlenen sınıf(lar)**  
    4) `grads = ∂score/∂conv_out` → Kanal başına ortalama al (`weights`)  
    5) `cam = ReLU(sum(weights * conv_out_channels))` → [0,1] normalize  
    6) Orijinal resme **ısı haritasını** `alpha` ile yarı saydam bindir.
  - Dönen çıktı: `(orig_uint8_img, heatmap_overlay_uint8)` listesi.

### Parametreler & İpuçları
- `class_idx` : Hedef sınıfı kontrol etmek için kullanılır; **yanlış sınıfa bakmayı** önlemek adına faydalıdır.
- `alpha` : Isı haritası şeffaflığı; 0.25–0.5 arası tipik.
- Base/Head bağımlılığı: Eğer `grads is None` → head zinciri base çıktısına **bağlı değildir** (mimariyi kontrol edin).
- Görsel aralığı: `_to_uint8_rgb` her türlü `[−1,1]`, `[0,1]`, `[0,255]` girişini güvenli biçimde `uint8 [0,255]`’e çevirir.

In [ ]:
def _apply_head_from_base_features(feats, model, base):
    """
    feats: (1, Hc, Wc, C)  base(x) çıktısı
    model: tam model (Input -> base -> GAP -> Dropout -> Dense)
    base : model içindeki conv gövde (ör. model.get_layer('efficientnetb0'))
    """
    x = feats
    take = False
    for ly in model.layers:
        if ly.name == base.name and not take:
            take = True
            continue
        if not take:
            continue
        if isinstance(ly, tf.keras.layers.InputLayer):
            continue
        x = ly(x, training=False)
    return x

def grad_cam_batch(x_imgs, class_idx=None, alpha=0.35):
    outs = []
    B = int(x_imgs.shape[0])

    for b in range(B):
        x1 = x_imgs[b:b+1]

        with tf.GradientTape() as tape:
            conv_out = base(x1, training=False)   # (1,Hc,Wc,C)
            tape.watch(conv_out)

            preds = _apply_head_from_base_features(conv_out, model, base)
            if preds.shape.rank is None or preds.shape.rank > 2:
                xgap = tf.reduce_mean(conv_out, axis=[1, 2])
                drop_ly = None
                dense_ly = None
                
                for ly in reversed(model.layers):
                    if isinstance(ly, tf.keras.layers.Dense) and dense_ly is None:
                        dense_ly = ly
                    elif isinstance(ly, tf.keras.layers.Dropout) and drop_ly is None:
                        drop_ly = ly
                    if drop_ly is not None and dense_ly is not None:
                        break
                xhead = xgap
                if drop_ly is not None:
                    xhead = drop_ly(xhead, training=False)
                if dense_ly is not None:
                    preds = dense_ly(xhead, training=False)
                else:
                    raise RuntimeError("Head katmanları bulunamadı: Dense katmanı sonda bekleniyordu.")

            # Hedef sınıf
            if class_idx is None:
                target_id = tf.argmax(preds[0], axis=-1)   # sınıf ekseni boyunca
            else:
                target_id = class_idx if isinstance(class_idx, int) else class_idx[b]
                target_id = tf.convert_to_tensor(target_id)

            score = preds[0, target_id]

        grads = tape.gradient(score, conv_out)
        if grads is None:
            raise RuntimeError("Grad alınamadı: head zinciri base çıktısına bağlı değil.")

        conv  = conv_out[0]                        
        grad  = grads[0]                           
        weights = tf.reduce_mean(grad, axis=(0,1), keepdims=True)
        cam = tf.reduce_sum(weights * conv, axis=-1)  
        cam = tf.nn.relu(cam).numpy()
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-12)

        # Görselleştirme
        img = _to_uint8_rgb(x_imgs[b])
        heat = cv2.resize(cam.astype(np.float32), (img.shape[1], img.shape[0]))
        heat = (255 * heat).astype(np.uint8)
        heat = cv2.applyColorMap(heat, cv2.COLORMAP_JET)
        heat = cv2.cvtColor(heat, cv2.COLOR_BGR2RGB)
        vis  = cv2.addWeighted(heat, alpha, img, 1 - alpha, 0)
        outs.append((img, vis))
    return outs
    
for xb, yb in test_ds.take(1):
    k = min(5, xb.shape[0])
    res_eigen = eigen_cam_batch(xb[:k], alpha=0.35)   # mevcut çalışan
    res_grad  = grad_cam_batch (xb[:k], alpha=0.35)   # yeni

    for i in range(k):
        img_e, vis_e = res_eigen[i]
        img_g, vis_g = res_grad[i]
        plt.figure(figsize=(9,3))
        plt.subplot(1,3,1); plt.imshow(img_e); plt.axis('off'); plt.title('Input')
        plt.subplot(1,3,2); plt.imshow(vis_e); plt.axis('off'); plt.title('Eigen-CAM')
        plt.subplot(1,3,3); plt.imshow(vis_g); plt.axis('off'); plt.title('Grad-CAM')
        plt.tight_layout(); plt.show()


## 16) CAM Görsellerinden Seçki (2 Doğru + 2 Yanlış)

Bu hücre, test kümesinden **doğru sınıflandırılmış** ve **yanlış sınıflandırılmış** örneklerden seçerek **Eigen-CAM / Grad-CAM** bindirmelerini yan yana gösterir.

**Fonksiyon**
- `show_cam_samples_from_ds(test_ds, y_true_vec, y_pred_vec, classes, eigen_cam_fn, grad_cam_fn, alpha=0.35, k_correct=2, k_wrong=2)`
  - **Toplama**: `test_ds` içindeki tüm görüntüler belleğe alınır (`x_all`).  
  - **Seçim**:  
    - `correct_idx`: `y_true_vec == y_pred_vec` (doğru tahminler)  
    - `wrong_idx`  : `y_true_vec != y_pred_vec` (yanlış tahminler)  
    - İlk `k_correct` ve `k_wrong` örnek alınır.  
  - **CAM üretimi**:  
    - `eigen_cam_fn(xb_tf, alpha)` → `(img, vis)`  
    - `grad_cam_fn(xb_tf, alpha)`  → `(img, vis)`  
    - Her örnek için Input + (varsa) Eigen-CAM + Grad-CAM sütunları çizilir.
  - **Başlıklar**: `T:{true_c} / P:{pred_c}` ile gerçek/tahmin edilen sınıflar gösterilir.

**Önemli Notlar**
- Fonksiyon, CAM için `tf.float32` tensörüne dönüştürür (**`xb_tf`**).  
- Görüntü çiziminde `plt.imshow(np.uint8(xb[0]))` kullanılıyor.  
  - Eğer pipeline’da görüntüler **[-1, 1]** aralığındaysa, orijinal input’u doğru görmek için **önce** `[0,255]` aralığına ölçeklemek gerekebilir (aksi halde soluk/bozuk görünebilir).  
  - CAM bindirmeleri (`vis`) zaten `uint8` ve doğru aralıktadır (Eigen-CAM/Grad-CAM içinde `_to_uint8_rgb` kullanılıyor).


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

def show_cam_samples_from_ds(test_ds, y_true_vec, y_pred_vec, classes,
                             eigen_cam_fn=None, grad_cam_fn=None,
                             alpha=0.35, k_correct=2, k_wrong=2):
    """
    test_ds     : tf.data.Dataset -> (images, labels)  [labels one-hot olabilir]
    y_true_vec  : 1D numpy array, gerçek sınıf indeksleri
    y_pred_vec  : 1D numpy array, tahmin sınıf indeksleri
    classes     : sınıf isim listesi
    """
   
    x_batches = []
    for x, _ in test_ds:
        x_batches.append(x.numpy())
    x_all = np.concatenate(x_batches, axis=0)

    correct_idx = np.where(y_true_vec == y_pred_vec)[0]
    wrong_idx   = np.where(y_true_vec != y_pred_vec)[0]

    pick_correct = correct_idx[:k_correct]
    pick_wrong   = wrong_idx[:k_wrong]
    chosen = list(pick_correct) + list(pick_wrong)

    for i in chosen:
        xb = x_all[i:i+1]
        row_imgs = []

        if eigen_cam_fn is not None:
            xb_tf = tf.convert_to_tensor(xb, dtype=tf.float32)  
            img_e, vis_e = eigen_cam_fn(xb_tf, alpha=alpha)[0]
            row_imgs.append(("Eigen-CAM", vis_e))
        
        if grad_cam_fn is not None:
            xb_tf = tf.convert_to_tensor(xb, dtype=tf.float32)  
            img_g, vis_g = grad_cam_fn(xb_tf, alpha=alpha)[0]
            row_imgs.append(("Grad-CAM", vis_g))

        cols = len(row_imgs) + 1
        plt.figure(figsize=(4*cols, 4))
        # Orijinal görüntü
        plt.subplot(1, cols, 1)
        plt.imshow(np.uint8(xb[0]))
        plt.axis('off')
        true_c = classes[int(y_true_vec[i])]
        pred_c = classes[int(y_pred_vec[i])]
        plt.title(f"Input\nT:{true_c} / P:{pred_c}")

        for j, (name, overlay) in enumerate(row_imgs, start=2):
            plt.subplot(1, cols, j)
            plt.imshow(overlay)
            plt.axis('off')
            plt.title(name)

        plt.tight_layout()
        plt.show()

show_cam_samples_from_ds(
    test_ds,
    y_true_vec,
    y_pred_vec,
    classes,
    eigen_cam_fn=eigen_cam_batch,
    grad_cam_fn=grad_cam_batch,
    alpha=0.35
)


## 17) Tek Görselden Tahmin (predict_file)

Bu fonksiyon, **kaydedilmiş modeli kullanarak tek bir resim için sınıf tahmini yapar**.

**Adımlar**
- `Image.open(img_path).convert("RGB").resize((IMG_SIZE, IMG_SIZE))`  
  → Resmi RGB formatına çevirip modelin beklediği boyuta getirir.
- `np.array(img).astype("float32")`  
  → Görüntüyü sayısal tensöre dönüştürür.
- `preprocess_input(x)`  
  → Piksel değerlerini **[-1,1]** aralığına normalize eder.
- `model.predict(x[None, ...])`  
  → Tek resmi batch boyutuna genişleterek tahmin eder.
- `argsort()[::-1][:topk]`  
  → En yüksek olasılıklı **ilk `topk` sınıfı** seçer (varsayılan `topk=3`).


In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input
from PIL import Image
import numpy as np, tensorflow as tf

def predict_file(img_path, topk=3):
    img = Image.open(img_path).convert("RGB").resize((IMG_SIZE, IMG_SIZE))
    x   = np.array(img).astype("float32")
    x   = preprocess_input(x)                      # [-1,1]
    pr  = model.predict(x[None, ...], verbose=0)[0]
    idx = pr.argsort()[::-1][:topk]
    return [(classes[i], float(pr[i])) for i in idx]

preds = predict_file("/kaggle/input/flowers-recognition/flowers/rose/10090824183_d02c613f10_m.jpg")
for cls, prob in preds:
    print(f"{cls}: {prob:.2%}")
   


## 18) Test-Time Augmentation (TTA) ile Tahmin

Bu adım, **tahmin sırasında veri artırma** (örn. yatay çevirme) uygulayarak modelin genelleme gücünü artırır ve **daha kararlı sonuçlar** elde eder.

### Fonksiyon
- `predict_with_tta(x_batch, n=4)`
  - `x_batch` : Önceden `preprocess_input` uygulanmış görüntü batch’i.
  - `n` : Aynı batch için kaç farklı görünüm (augmentation) ile tahmin alınacağı.
  - Döngü:
    - Çift turlar → orijinal görüntü  
    - Tek turlar → **yatay çevrilmiş** görüntü (`tf.image.flip_left_right`).
    - `model.predict` çıktıları `outs` listesine eklenir.
  - `np.mean(outs, axis=0)` → Tüm tahminlerin ortalaması alınır (**olasılıkları yumuşatır**).

In [ ]:
def predict_with_tta(x_batch, n=4):  
    outs = []
    for t in range(n):
        xb = x_batch
        if t % 2 == 1:
            xb = tf.image.flip_left_right(xb)
        pr = model.predict(xb, verbose=0)
        outs.append(pr)
    return np.mean(outs, axis=0)


avg_probs = np.vstack([predict_with_tta(xb) for xb, _ in test_ds])
y_pred = avg_probs.argmax(1)
tta_acc = (y_true_vec == y_pred).mean()


idx = 0   # test setindeki ilk örnek
print("Gerçek sınıf:", classes[y_true_vec[idx]])
print("Tahmin edilen:", classes[y_pred[idx]])
print("Ortalama olasılıklar:", avg_probs[idx])



## 19) Bottleneck (Önceden Çıkarılmış Özellik) Yaklaşımı – Hızlı Arama

Bu blok, EfficientNetB0’un konvolüsyonel kısmını **donuk (frozen) özellik çıkarıcı** olarak kullanıp, eğitim verisini **bir kez ileri geçirerek** (forward) **bottleneck** özelliklerini RAM’e (veya dosyaya) alır. Amaç: **deneme/arama hızını** büyük ölçüde artırmak.

**Ne yapar?**
- `IMG_SIZE` hem `int` hem `(H,W)` gelirse uyumludur.
- `feature_extractor`: `EfficientNetB0(include_top=False)` + `GlobalAveragePooling2D`  
  → Sadece **özellik** üretir, **eğitilmez** (`trainable=False`).
- `subset_frac`: Hızlı denemeler için eğitim/validasyonun bir **alt kısmını** kullanır (örn. 0.3).
- `ds_to_features(ds)`:  
  - Her batch’i `feature_extractor` içinden geçirip `(B, C)` boyutlu özellikleri toplar.  
  - Etiketleri de toplar; **one-hot** ise **sparse** sınıf indeksine çevirir.  
- `np.savez(...)`: Özellikleri ve etiketleri diske kaydeder (isteğe bağlı).

**Çıktılar**
- `Xtr, ytr`: Eğitim özellikleri ve etiketler  
- `Xva, yva`: Validasyon özellikleri ve etiketler  
- Örnek: `Train feats: (N_train, C)  Val feats: (N_val, C)`

**Neden faydalı?**
- Üstte küçük bir **head** (Dense + Dropout vb.) eğiterek **çok hızlı** model araması (LR, dropout, hidden size...) yapabilirsiniz.
- Ağır konvolüsyon kısmı her seferinde çalışmadığından **GPU/CPU süresi** kısalır.


> Özet: Bottleneck yaklaşımı, **hızlı prototipleme ve hiperparametre araması** için idealdir; sonrasında dilerseniz tam eğitim (end-to-end veya kademeli fine-tuning) ile sonuçları rafine edebilirsiniz.


In [ ]:
import tensorflow as tf, numpy as np, os, pandas as pd
from tensorflow.keras import layers, models


if isinstance(IMG_SIZE, tuple):
    H, W = IMG_SIZE
else:
    H = W = int(IMG_SIZE)

num_classes = len(classes)
inp = tf.keras.Input(shape=(H, W, 3))
b0  = tf.keras.applications.EfficientNetB0(include_top=False, weights="imagenet")(inp, training=False)
gap = layers.GlobalAveragePooling2D()(b0)
feature_extractor = models.Model(inp, gap, name="b0_gap")
feature_extractor.trainable = False

subset_frac = 1.0  
def take_fraction(ds, frac):
    if frac >= 1.0: return ds
    total = 0
    for _ in ds: total += 1
    take_n = max(1, int(total*frac))
    return ds.take(take_n)

train_search = take_fraction(train_ds, subset_frac)
val_search   = take_fraction(val_ds,   subset_frac)

def ds_to_features(ds):
    X, y = [], []
    for xb, yb in ds:
        feats = feature_extractor(xb, training=False).numpy()  # (B, C)
        X.append(feats)
        y.append(yb.numpy())
    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)

    if y.ndim == 2 and y.shape[1] == num_classes:
        y = y.argmax(axis=1)
    return X, y.astype("int32")

Xtr, ytr = ds_to_features(train_search)
Xva, yva = ds_to_features(val_search)

print("Train feats:", Xtr.shape, "Val feats:", Xva.shape)
np.savez("/kaggle/working/bottlenecks.npz", Xtr=Xtr, ytr=ytr, Xva=Xva, yva=yva)


## 20) Hızlı Hiperparametre Araması (Optimizer + Batch Size Dahil)

Bu hücre, **bottleneck özellikleri** üzerinde yalnızca **üst sınıflandırıcı (head)** için  
hızlı ve geniş kapsamlı bir **hiperparametre araması** yapar.  
Önceki bloklarda oluşturulan `Xtr, ytr, Xva, yva` verilerini kullanır.

### Adımlar

1. **Bottleneck Özellik Çıkarımı (subset_frac=0.5)**  
   - Eğitim ve doğrulama verilerinin %50’si alınır.  
   - EfficientNetB0 gövdesi **donuk** bırakılır.  
   - Böylece veri boyutu yarıya iner ve deneyler çok daha hızlı tamamlanır.

2. **Head Modeli Kurulumu ve Eğitimi**  
   - Mimari: `Dense(dense_units, relu)` (opsiyonel) → `Dropout(drop)` → `Dense(num_classes, softmax)`.
   - Kayıp fonksiyonu: `SparseCategoricalCrossentropy`.  
   - Metrik: `accuracy`.
   - Erken durdurma (`EarlyStopping`) ile aşırı eğitim engellenir.

3. **Grid Araması**  
   - Öğrenme oranı (`lr`): **1e-3**, **5e-4**  
   - Dropout (`drop`): **0.3**, **0.5**  
   - Gizli katman nöron sayısı (`dense_units`): **0**, **128**  
   - Optimizasyon algoritması (`optimizer`): **adam**, **rmsprop**  
   - Batch size (`batch_size`): **32**, **64**  
   - Toplam **24 kombinasyon** denenir. 

4. **Raporlama ve Kayıt**  
   - Tüm denemeler `df` DataFrame’inde saklanır, **val_acc**’e göre sıralanır.
   - Sonuçlar `/kaggle/working/results_hparam_search.csv` dosyasına kaydedilir.
   - En iyi kombinasyon ve en yüksek `val_acc` konsola yazdırılır.


In [ ]:
import tensorflow as tf, numpy as np, pandas as pd
from tensorflow.keras import layers, models, optimizers, losses, callbacks


if isinstance(IMG_SIZE, tuple):
    H, W = IMG_SIZE
else:
    H = W = int(IMG_SIZE)

num_classes = len(classes)
inp = tf.keras.Input(shape=(H, W, 3))
b0  = tf.keras.applications.EfficientNetB0(include_top=False, weights="imagenet")(inp, training=False)
gap = layers.GlobalAveragePooling2D()(b0)
feature_extractor = models.Model(inp, gap, name="b0_gap")
feature_extractor.trainable = False


subset_frac = 0.5
def take_fraction(ds, frac: float):
    if frac >= 1.0:
        return ds
    total = 0
    for _ in ds:
        total += 1
    take_n = max(1, int(total * frac))
    return ds.take(take_n)

train_search = take_fraction(train_ds, subset_frac)
val_search   = take_fraction(val_ds,   subset_frac)

def ds_to_features(ds):
    X, y = [], []
    for xb, yb in ds:
        feats = feature_extractor(xb, training=False).numpy() 
        X.append(feats)
        y.append(yb.numpy())
    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    if y.ndim == 2 and y.shape[1] == num_classes: 
        y = y.argmax(axis=1)
    return X, y.astype("int32")

Xtr, ytr = ds_to_features(train_search)
Xva, yva = ds_to_features(val_search)
print(f"[Hızlı Mod] Train feats: {Xtr.shape}, Val feats: {Xva.shape} (subset_frac={subset_frac})")
early_stop = callbacks.EarlyStopping(monitor="val_accuracy", patience=2, restore_best_weights=True)

def make_optimizer(name: str, lr: float):
    name = (name or "adam").lower()
    if name == "adam":
        return optimizers.Adam(learning_rate=lr)
    if name == "rmsprop":
        return optimizers.RMSprop(learning_rate=lr)
    if name == "sgd":
        return optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)
    raise ValueError(f"Bilinmeyen optimizer: {name}")

def quick_try_head(
    lr=1e-3,
    drop=0.4,
    dense_units=0,
    optimizer_name="adam",
    batch_size=64,
    epochs=8,        
    verbose=0,
):
    inputs = layers.Input(shape=(Xtr.shape[1],))
    x = inputs
    if dense_units > 0:
        x = layers.Dense(dense_units, activation="relu")(x)
        x = layers.Dropout(drop)(x)
    else:
        x = layers.Dropout(drop)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    m = models.Model(inputs, outputs)
    m.compile(optimizer=make_optimizer(optimizer_name, lr),
              loss=losses.SparseCategoricalCrossentropy(),
              metrics=["accuracy"])

    h = m.fit(
        Xtr, ytr,
        validation_data=(Xva, yva),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=verbose,
    )
    return float(max(h.history["val_accuracy"])), m


grid_lr   = [1e-3, 5e-4]
grid_drop = [0.3, 0.5]
grid_dense= [0, 128]
grid_opt  = ["adam", "rmsprop"]      
grid_bs   = [32, 64]

tries = []
best_cfg, best_acc, best_model = None, -1.0, None

for lr in grid_lr:
    for drop in grid_drop:
        for dense in grid_dense:
            for opt_name in grid_opt:
                for bs in grid_bs:
                    acc, model_head = quick_try_head(
                        lr=lr,
                        drop=drop,
                        dense_units=dense,
                        optimizer_name=opt_name,
                        batch_size=bs,
                        epochs=8,   
                        verbose=0,
                    )
                    tries.append((lr, drop, dense, opt_name, bs, acc))
                    if acc > best_acc:
                        best_cfg, best_acc, best_model = (lr, drop, dense, opt_name, bs), acc, model_head

cols = ["lr", "dropout", "dense_units", "optimizer", "batch_size", "val_acc"]
df = pd.DataFrame(tries, columns=cols).sort_values("val_acc", ascending=False)
df.to_csv("/kaggle/working/results_hparam_search.csv", index=False)
print("Tablo kaydedildi")


## 21) Hiperparametre Arama Sonuçlarının Tabloya Dökülmesi

Bu hücre, `tries` listesindeki tüm denemeleri **DataFrame**’e çevirir, en iyi doğrulamaya göre sıralar ve hem **CSV** hem **Markdown** biçiminde çıktılar.

**Adımlar**
- `tries` içeriğine göre sütun adlarını otomatik belirler:  
  - 3 elemanlı tuple → `["learning_rate", "dropout", "val_acc"]`  
  - 6 elemanlı tuple → `["lr", "dropout", "dense_units", "optimizer", "batch_size", "val_acc"]`
- `df.sort_values("val_acc", ascending=False)` → En yüksek doğruluk üste alınır.
- `df.to_csv("/kaggle/working/results_hparam_search.csv")` → Sonuçlar **CSV** dosyası olarak kaydedilir.
- `df.to_markdown()` → Notebook çıktısında **Markdown tablosu** halinde gösterir.  
  - `tabulate` yüklü değilse fallback olarak `tabulate()` ile basit biçimde basar.


In [ ]:
import pandas as pd

if not tries:
    raise ValueError("tries boş görünüyor.")

n = len(tries[0])
if n == 3:
    cols = ["learning_rate", "dropout", "val_acc"]
elif n == 6:
    cols = ["lr", "dropout", "dense_units", "optimizer", "batch_size", "val_acc"]
else:
    cols = [f"col{i}" for i in range(n)]  
df = pd.DataFrame(tries, columns=cols)
df = df.sort_values("val_acc", ascending=False)
df.to_csv("/kaggle/working/results_hparam_search.csv", index=False)

try:
    print(df.to_markdown(index=False))
except Exception:
    from tabulate import tabulate 
    print(tabulate(df, headers='keys', tablefmt='github', showindex=False))

## 22) En İyi Hiperparametre Seti ve Kısa Gerekçe

Bu hücre, hiperparametre aramasında elde edilen en iyi kombinasyonu ve kısa bir seçim gerekçesini otomatik olarak raporlar.

**Adımlar**
- `best_idx = df["val_acc"].idxmax()` → En yüksek doğrulama doğruluğuna sahip satırın indeksini bulur.
- `best = df.loc[best_idx].to_dict()` → Bu satırdaki tüm hiperparametreleri (ör. lr, dropout, dense_units…) sözlük olarak alır.
- `print` döngüsü ile her hiperparametre ve değeri satır satır gösterir.

**Gerekçe Mantığı**
- En yüksek `val_acc` değeri temel alınır.
- İlk 3 sonucu inceler; eğer 1. ve 2. skorlar **çok yakınsa (<0.005 fark)**:
  - Dropout veya L2 gibi **düzenlileştirme** parametreleri karşılaştırılır.
  - Daha yüksek düzenlileştirme varsa **daha stabil ve genelleştirilebilir** yapı vurgulanır.
- Fark belirgin ise: “Alternatiflere göre belirgin şekilde daha yüksek doğrulama başarımı” notu eklenir.


In [ ]:

import numpy as np

assert "val_acc" in df.columns, "df içinde 'val_acc' sütunu yok!"
best_idx = df["val_acc"].idxmax()
best = df.loc[best_idx].to_dict()

print("Seçilen en iyi set:")
for k, v in best.items():
    print(f" - {k}: {v}")

top3 = df.sort_values("val_acc", ascending=False).head(3).reset_index(drop=True)
msg = [f"\nGerekçe:",
       f"- En yüksek doğrulama başarımı: {top3.loc[0,'val_acc']:.4f}."]

if len(top3) > 1 and (top3.loc[0,"val_acc"] - top3.loc[1,"val_acc"]) < 0.005:
    hint_cols = [c for c in df.columns if ("dropout" in c.lower()) or ("l2" in c.lower())]
    if hint_cols:
        better_reg = []
        for c in hint_cols:
            v0 = top3.loc[0, c]
            v1 = top3.loc[1, c]
            if isinstance(v0, (int,float)) and isinstance(v1, (int,float)) and v0>=v1:
                better_reg.append(f"{c}={v0}")
        if better_reg:
            msg.append(f"- Yakın rakibe kıyasla daha güçlü düzenlileştirme ({', '.join(better_reg)}) içeriyor.")
    msg.append("- Skorlar çok yakın olduğu için daha **stabil** ve **genelleştirilebilir** yapı tercih edildi.")
else:
    msg.append("- Alternatiflere göre belirgin şekilde daha yüksek doğrulama başarımı.")

print("\n".join(msg))


## 23) Yanlış Sınıflandırılan Örnekleri Görselleştirme

Bu hücre, **test kümesinde modelin yanıldığı ilk 15 örneği** doğru/güvenli bir görüntü ölçeklemesiyle çizip kaydeder.

**Ne yapıyor?**
- `to_uint8_rgb(x)`  
  - Görüntüyü olası aralıklardan (`[-1,1]`, `[0,1]`, `[0,255]`) **güvenle** `uint8 [0,255]` aralığına çevirir.  
  - CAM görselleriyle tutarlı ve “doğal renkli” görüntü sunar.
- `y_true` / `y_pred`  
  - Test kümesinden **gerçek etiketleri** (one-hot’tan `argmax`) ve **tahminleri** üretir.  
  - `mis_idx` ile **yanlış tahmin edilen** indeksler bulunur.
- Görsellerin derlenmesi  
  - `test_ds` içindeki tüm batch’lerdeki görüntüler `to_uint8_rgb` ile normalize edilip `test_images` listesine eklenir.
- Çizim ve kayıt  
  - `matplotlib` ile 3×5 düzeninde **ilk 15 yanlış örnek** çizilir.  
  - Başlık: `T:{gerçek sınıf} / P:{tahmin edilen sınıf}`  
  - Dosya kaydı: `/kaggle/working/misclassified_examples.png`

**Neden önemli?**
- Modelin **hangi sınıflarda ve hangi tür görsellerde yanıldığı** hızla görülür.  
- Karışan sınıfları inceleyip **augmentation** stratejisi, sınıf ağırlıkları veya veri kalitesi hakkında iyileştirme kararları alabilirsiniz.


In [ ]:
import numpy as np, matplotlib.pyplot as plt

def to_uint8_rgb(x):
    """
    x: np.ndarray or tf.Tensor, shape (H,W,3)
       Olası aralıklar: [-1,1] ya da [0,1] ya da [0,255]
    return: np.uint8 in [0,255]
    """
    if hasattr(x, "numpy"):
        x = x.numpy()
    x = x.astype(np.float32)
    vmin, vmax = x.min(), x.max()
    if vmax <= 1.0 and vmin >= -1.0:   # [-1,1] veya [0,1]
        if vmin < 0:                   # [-1,1]
            x = (x + 1.0) * 127.5
        else:                          # [0,1]
            x = x * 255.0
    x = np.clip(x, 0, 255).astype(np.uint8)
    return x

y_true_batches = []
for _, y in test_ds:
    y_true_batches.append(np.argmax(y.numpy(), axis=1))
y_true = np.concatenate(y_true_batches, axis=0)
y_pred_probs = model.predict(test_ds, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)
mis_idx = np.where(y_true != y_pred)[0]
test_images = []
for xb, _ in test_ds:
    imgs = np.stack([to_uint8_rgb(im) for im in xb]) 
    test_images.extend(imgs)

fig, axes = plt.subplots(3, 5, figsize=(12, 7))
for ax, i in zip(axes.ravel(), mis_idx[:15]):
    ax.imshow(test_images[i])
    ax.set_title(f"T:{classes[y_true[i]]}\nP:{classes[y_pred[i]]}")
    ax.axis('off')
plt.tight_layout()
plt.savefig("/kaggle/working/misclassified_examples.png")
plt.show()
